#      CLOUD DATA ENGINEERING MODULE BY BILAL ZAFAR 

Import necessary Libraries

In [2]:
import requests
from io import StringIO
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from datetime import datetime
from icecream import ic


Step 0: Maintaining a Log File



This step is done to record the logs while performing ETL and it is not neccessary in an ETL Pipeline

In [3]:
def log_file(message):
    with open("logging_data_file.txt", "a") as f:
        f.write(f"{datetime.now()}:{message}\n")
        

Step 1: Extract

In [5]:

def extraction(url, table_attribute):
    # Step 1: Fetch the HTML content from the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")  # Corrected typo in "html.parser"

    # Step 2: Find the table element using the specified attribute
    table = soup.find("span", string=table_attribute)

    # Step 3: Read the HTML table into a DataFrame using pandas
    if table:
        df = pd.read_html(str(table))[0]
        return df
    else:
        print(f"No table found with attribute '{table_attribute}' on the page.")
        return None



Step 2 : TRANSFORMATION

In [ ]:
def transform(df, csv_path):
    """ This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies"""

    exchange_rate = pd.read_csv(csv_path, index_col=0).to_dict()[Rate]

    df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['GBP'], 2)
    df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['EUR'], 2)
    df['MC_INR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['INR'], 2)

    ic(df['MC_EUR_Billion'][4])

    log_progress('Data transformation complete. Initiating Loading process')

    return df


Step 3 : LOADING BY CSV

In [1]:
def load_to_csv(df, output_path):
    """ This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing."""

    df.to_csv(output_path)

    log_progress('Data saved to CSV file')

Loading data to SQL




In [2]:
def load_to_db(df, sql_connection, table_name):
    """ This function saves the final data frame to a database
    table with the provided name. Function returns nothing."""

    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

    log_progress('Data loaded to Database as a table, Executing queries')


def run_query(query_statement, sql_connection):
    """ This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. """

    cursor = sql_connection.cursor()
    cursor.execute(query_statement)
    result = cursor.fetchall()

    log_progress('Process Complete')

    return result

Executing Pipeline

In [4]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Largest_banks_data.csv'
    database_name = 'Banks.db'
    table_name = 'Largest_banks'
    #
    log_progress('Preliminaries complete. Initiating ETL process')
    #

    df = ic(extract(url, 'By market capitalization'))

    transform(df, './data/exchange_rate.csv')

    load_to_csv(df, output_csv_path)

    with sqlite3.connect(database_name) as conn:
        load_to_db(df, conn, table_name)

        ic(run_query('SELECT * FROM Largest_banks', conn))

        ic(run_query('SELECT AVG(MC_GBP_Billion) FROM Largest_banks', conn))

        ic(run_query('SELECT "Bank name" FROM Largest_banks LIMIT 5', conn))

NameError: name 'log_progress' is not defined